# 1. CoLLiE 的 安装使用

> 1.1 &ensp; CoLLiE 的 快速安装
> 
> 1.2 &ensp; CoLLiE 的 快速上手
> 
> 1.3 &ensp; CoLLiE 的 快速运行

&ensp; &ensp; CoLLie作为一款端到端的大语言模型高度集成化训练框架，其功能主要体现在以下的四个方面，相关内容在后续的Tutorial中会有详细的介绍：

&emsp; &emsp; 首先，CoLLiE 致力于涵盖包含适用于各种特异场景的并行方式，整合四种并行策略：**数据并行 DP**、**流水线并行 PP**、**张量并行 TP**（ 以及 **3D并行**）、**零冗余优化器 ZeRO**（[tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-6-parallel.html)）

&emsp; &emsp; 其次，CoLLiE 致力于涵盖大语言模型从预训练、全量微调、参数高效微调的全流程，提出面向全参数高效微调的**LOMO算法**，支持**LoRA**、**PrefixTuning**、**PromptTuning**等参数高效微调方案（[tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-5-lomo-peft.html)）

&emsp; &emsp; 再次，CoLLiE 致力于涵盖从数据准备、数据处理、模型训练、模型评测、模型部署的训练整体环节：通过**Trainer**、**Config**模块，轻松上手LLM微调实战（[tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-basic.html)），通过**Callback**、**Monitor**模块，全方位掌控LLM微调过程（[tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-4-advanced.html)）

&emsp; &emsp; 最后，CoLLiE 致力于与时俱进，不断集成最新的模型架构、优化框架，兼容**flash_attn**，极大提升Transformer运算效率，在有限的资源上让大模型轻松驾驭长序列（[tutorial-2](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-2-feature.html)）

<iframe width="560" height="315" src="https://www.youtube.com/embed/HIb910e4IEA?si=uC8YCQ7_4S1F06Au" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

## 1.1 &ensp; CoLLiE 的 快速安装

&ensp; &ensp; CoLLiE通过git结合pip install安装，只需要一行指令如下。

```sh
pip install git+https://github.com/OpenLMLab/collie.git
```

&ensp; &ensp; CoLLiE在安装时需要保证保证环境已有的torch对应cuda版本和nvcc -V结果一致，例如笔者的nvcc是11.7版本，对应torch使用1.13.1+cu117

``` bash
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
```

## 1.2 &ensp; CoLLiE 的 快速上手

&ensp; &ensp; 在这一节中，我们将给出一个完整的 CoLLiE 使用代码案例，由此，一方面是方便大家快速的跑通 CoLLiE、用上 CoLLiE，实现真正的 CoLLiE 快速上手；另一方面也是借助这个 CoLLiE 的使用案例，向大家介绍 CoLLiE 自定义的一些基础模块包括 Config、Dataset、Evaluator、Trainer；接着在后续小节中，我们将详细介绍各个模块的运行原理的使用细节。本教程使用的CoLLiE使用案例是，使用 Adam（优化算法）ZeRO3（并行策略）全参数训练 LLaMA2（基座模型），针对 Alpaca数据（训练数据）进行指令微调（微调任务）。首先导入一些必要的包，并且设置加载模型的路径或名称。

``` python
import torch
from datasets import load_dataset
from transformers import LlamaTokenizer, get_linear_schedule_with_warmup

from collie import CollieConfig, Trainer
from collie import LRMonitor, LossMonitor
from collie import CollieDatasetForTraining
from collie import LlamaForCausalLM

model_name_or_path = 'meta-llama/Llama-2-7b-hf'
```

&ensp; &ensp; 然后就到了CoLLiE最关键的一个步骤，即定制CoLLiE的配置类CollieConfig。这里首先通过从模型路径中加载初始化配置类，这时加载出来的CollieConfig，会自带一些模型的基本超参数，例如层数、维度数量等。紧接着，对于初始化好的CollieConfig，我们可以设置不同并行策略的超参数，例如数据并行、流水线并行、张量并行的数量（对应 config.dp_size、config.pp_size、config.tp_size），以及训练的其他超参数，例如训练轮数、batch大小等，还有deepspeed相关参数，例如ZeRO策略、是否开启BF16混合精度、训练监控方式等；关于配置类，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-basic.html#3.1-%E2%80%82-CoLLiE-的-Config-模块)。

``` python
config = CollieConfig.from_pretrained(model_name_or_path)

config.dp_size = 4
config.pp_size = 1
config.tp_size = 1
config.train_epochs = 3
config.train_micro_batch_size = 32
config.gradient_accumulation_steps = 1
config.ds_config = {
    "fp16": {"enabled": True},
    "monitor_config": {
        "enabled": True,
        "tag": f"tutorial-lr_0.00001-epoch_3",
        "wandb": {
            "enabled": True,
            "team": "collie_exp",
            "project": "llama_alpaca",
            "group": f"llama2_7b",
        }
    },
    "zero_optimization": {"stage": 3},
}
```

&ensp; &ensp; 接下来是初始化 tokenizer、dataset、model、optimizer、lr_scheduler 。CoLLiE兼容huggingface或torch实现的上述内容，例如本例中使用的 tokenizer、lr_scheduler 从huggingface中导入，optimizer从torch中导入；关于CoLLiE自定义的高效微调优化器，详见 [tutorial-5](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-5-lomo-peft.html#5.1-%E2%80%82-CoLLiE-实现-LOMO-算法)。

&ensp; &ensp; 书归正传，此处使用的 dataset、model 是基于CoLLiE自定义的 CollieDataset（CollieDatasetForTraining）、CollieModelForCausalLM（LlamaForCausalLM）。其中，CollieDatasetForTraining通过在构造阶段区分prompt和output部分（分别对应字段“input”和“output”）在模型训练时自动划分出梯度更新范围，LlamaForCausalLM通过CollieConfig自动完成初始化，并且通过给定的路径名称完成参数加载，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-basic.html#3.2-%E2%80%82-CoLLiE-的-Dataset-和-Model)。

``` python
tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path, padding_side="left")

data = [{
        "intput": sample["prompt"].split("### Response:\n")[0] + "### Response:\n",
        "output": sample["prompt"].split("### Response:\n")[1] + tokenizer.eos_token
    } for sample in load_dataset("crumb/stanford_alpaca_full_prompts")["train"]]
train_dataset = CollieDatasetForTraining(data, tokenizer=tokenizer, max_length=2048)

model = LlamaForCausalLM.from_pretrained(model_name_or_path, config)

optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

total_step = (len(train_dataset) * config.train_epochs) // 128
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(total_step * 0.03),
    num_training_steps=total_step
)
```

&ensp; &ensp; 准备完数据、模型、优化器之后就可以开始训练了。CoLLiE使用Trainer模块集成整个训练过程，通过向Trainer传入模型model、配置类config、优化器optimizer、数据集train_dataset，通过trainer.train()即可快速开启训练过程，通过trainer.save_model()即可保存模型训练结果，保存格式和huggingface保存格式一致，可直接加载至相同模型的huggingface版本中；关于trainer的更多内容，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-trainer.html#3.4-%E2%80%82-CoLLiE-的-Trainer-模块)。

``` python
trainer = Trainer(
    model=model,
    config=config,
    optimizer=optimizer,
    train_dataset=train_dataset,
    monitors=[LRMonitor(config), LossMonitor(config)],
    lr_scheduler=lr_scheduler,
)

trainer.train()

trainer.save_model("./result")
```

&ensp; &ensp; 在这个过程中，为了显示训练过程中学习率和损失的变化过程，这里设置监控器列表monitors，包括实时输出当前学习率的LRMonitor 和 当前损失值的LossMonitor，两者都通过config初始化；更多关于monitor监控器的内容，详见 [tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-4-advanced.html#4.1-%E2%80%82-CoLLiE-的-Monitor-模块)。此外，在训练过程中，我们还允许加入评测相关的内容，对此CoLLiE通过评测单元Evaluator和对应评测指标Metric加以集成，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-basic.html#3.3-%E2%80%82-CoLLiE-的-Evaluator-和-Metric)。而在训练过程中，CoLLiE还可以根据回调单元Callback允许用户自定义模型训练过程中的行为，详见 [tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-4-advanced.html#4.2-%E2%80%82-CoLLiE-的-Callback-模块)。除了全参数训练之外，CoLLiE还支持参数高效的微调方法例如LoRA，详见 [tutorial-5](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-5-lomo-peft.html#5.2-%E2%80%82-CoLLiE-结合-PEFT-算法)；对应内存高效的全参数微调方法例如LoMO，详见 [tutorial-5](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-5-lomo-peft.html#5.1-%E2%80%82-CoLLiE-实现-LOMO-算法)。最后，CoLLiE还支持模型的部署，详见 [tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-4-advanced.html#4.3-%E2%80%82-CoLLiE-的-Server-模块)。

## 1.3 &ensp; CoLLiE 的 快速运行

&ensp; &ensp; 再将上述代码复制到train.py之后，通过以下的命令，我们就可以运行上述代码。

``` shell
CUDA_VISIBLE_DEVICES=0,1,2,3 torchrun --nnodes=1 --nproc_per_node=4 train.py
```

这个命令是基于touchrun的一个分布式运行，其中的参数如下表所示，通过该条命令我们在当前服务器的4个节点开启了4个子进程，从而执行4路数据并行的训练代码。如果想中止这条程序，只需要关闭其中一个子进程，剩余的程序就会自动结束。关于pytorch分布式相关的更多内容，可以参考 [pytorch官方文档](https://pytorch.org/docs/stable/index.html) 中 [关于torchrun的介绍](https://pytorch.org/docs/stable/elastic/run.html)。

| CUDA_VISIBLE_DEVICES=0,1,2,3 | 使用当前服务器上的0至3号显卡 |
| --- | --- |
| torchrun train.py | 以分布式的形式启动 train.py，即在0至3号显卡上并行运行train.py |
| --nnodes=1 | 在1台服务器（节点 node）上运行 |
| --nproc_per_node=4 | 当前服务器拥有的显卡数量（每个显卡对应一个进程 process） |

&ensp; &ensp; 在接下来的教程中，我们首先将介绍CoLLiE的功能特性，讲解应用CoLLiE解决给定任务，如何估算需要申请的计算资源数量；接着我们将一点点讲解前文所述的代码模块、并行策略、优化算法是什么，以及 CoLLiE 是如何实现它们的。